# Is this month's key phrase "Medium-Intensity Residential"?
> Recent sale prices and assessments by land use designation

- author: Josh Carp
- categories: [land-use]

In [1]:
#hide

# Hack: Ignore shapely/geos version mismatch warning
import warnings
warnings.simplefilter("ignore")

import pandas as pd
from google.cloud import bigquery

bq = bigquery.Client()

pd.set_option("display.max_rows", 500)

## Property transactions in Charlottesville: Hot market ~~for parcels designated as "Middle Intensity" in Future Land Use Plan~~

A response to [March 2022 property transactions in Charlottesville: Hot market for parcels designated as "Middle Intensity" in Future Land Use Plan](https://communityengagement.substack.com/p/march-2022-property-transactions)

I always make time to read Sean Tubbs's [newsletter](https://communityengagement.substack.com/), and I was interested to see what he has to say about the new "Medium-Intensity Residential" land use designation adopted in last year's [Comprehensive Plan](https://www.charlottesville.gov/DocumentCenter/View/7073/Comprehensive-Plan-Document---2021-1115-Final?bidId=). Sean noticed a number of properties with this designation, or located close to properties with this designation, that sold for high prices, some far above their most recent assessments. He [wrote](https://twitter.com/cvilletowncrier/status/1512836544486387720) that

> This month's key phrase is "Medium Intensity Residential".

and noted that

> ...properties with the Medium Intensity Residential designation are selling well over the 2022 assessment.

Sean writes that "I intend no analysis". Fair enough, but let's do some analysis! The city provides mostly current data on assessments, property transactions, and parcel details at its [Open Data Portal](https://opendata.charlottesville.org/), which I have been mirroring to a public dataset using [Google BigQuery](https://console.cloud.google.com/bigquery?dataset=cville_open_data&p=cvilledata&d=cville_open_data&page=dataset) for ease of use. Let's dig into the claims in the blog post. As I read it, there are two key arguments that we can evaluate:

- Parcels with the Medium-Intensity Residential designation have been selling for over their official assessments
- The difference between sale prices and assessments is greater for Medium-Intensity Residential than for other designations

First, let's compare median sale prices, assessments, and the ratio between the two across land use designations. I'm looking at all sales in 2022 to begin with. Note that I'm excluding sales with a price of $0 and sales that included multiple parcels.

In [2]:
#collapse-hide
bq.query(
    """
    select distinct
      percentile_cont(sales.saleamount, 0.5) over (partition by flum.desig_oct) as saleamount,
      percentile_cont(assessments.totalvalue, 0.5) over (partition by flum.desig_oct) as assessment,
      percentile_cont(sales.saleamount / assessments.totalvalue, 0.5) over (partition by flum.desig_oct) as saleassessmentratio,
      count(*) over (partition by flum.desig_oct) as count,
      flum.desig_oct as designation,
    from `cvilledata.cville_open_data.real_estate_sales` sales
    join `cvilledata.cville_open_data.real_estate_all_assessments` assessments on sales.parcelnumb = assessments.parcelnumb and assessments.taxyear = 2022
    join `cvilledata.cville_plans_together.flum_202110` flum on sales.parcelnumb = flum.parcelnumb
    where sales.saledate >= '2022-01-01'
      and sales.saleamount > 0
      and sales.bookpage not in (
        select
          bookpage
        from `cvilledata.cville_open_data.real_estate_sales`
        where saledate >= '2022-01-01'
          and saleamount > 0
        group by bookpage
        having count(*) > 1
      )
      and flum.desig_oct in (
        'General Residential',
        'Medium-Intensity Residential',
        'High-Intensity Residential'
      )
    order by count desc
    """
).result().to_dataframe()

,saleamount,assessment,saleassessmentratio,count,designation
0,423750.0,345100.0,1.128854,100,General Residential
1,355350.0,364200.0,1.054195,24,Medium-Intensity Residential
2,430000.0,440800.0,0.975198,2,High-Intensity Residential


As Sean noted, Medium-Intensity Residential parcels are selling for over their assessments, with the median ratio of sale price to assessed value at 1.05, meaning that sales were typically about 5% over assessments. However, General Residential parcels had an even higher ratio of 1.13, with the median parcel selling for about 13% more than its assessed value. But Sean only considered March; let's use that same filter to make sure we're looking at the same parcels:

In [3]:
#collapse-hide
bq.query(
    """
    select distinct
      percentile_cont(sales.saleamount, 0.5) over (partition by flum.desig_oct) as saleamount,
      percentile_cont(assessments.totalvalue, 0.5) over (partition by flum.desig_oct) as assessment,
      percentile_cont(sales.saleamount / assessments.totalvalue, 0.5) over (partition by flum.desig_oct) as saleassessmentratio,
      count(*) over (partition by flum.desig_oct) as count,
      flum.desig_oct as designation,
    from `cvilledata.cville_open_data.real_estate_sales` sales
    join `cvilledata.cville_open_data.real_estate_all_assessments` assessments on sales.parcelnumb = assessments.parcelnumb and assessments.taxyear = 2022
    join `cvilledata.cville_plans_together.flum_202110` flum on sales.parcelnumb = flum.parcelnumb
    where sales.saledate >= '2022-03-01' and sales.saledate < '2022-04-01'
      and sales.saleamount > 0
      and sales.bookpage not in (
        select
          bookpage
        from `cvilledata.cville_open_data.real_estate_sales`
        where saledate >= '2022-03-01' and saledate < '2022-04-01'
          and saleamount > 0
        group by bookpage
        having count(*) > 1
      )
      and flum.desig_oct in (
        'General Residential',
        'Medium-Intensity Residential',
        'High-Intensity Residential'
      )
    order by count desc
    """
).result().to_dataframe()

,saleamount,assessment,saleassessmentratio,count,designation
0,520000.0,412400.0,1.165224,43,General Residential
1,421000.0,396300.0,1.115945,14,Medium-Intensity Residential


Looking at March alone, Medium-Intensity Residential parcels had sale prices exceeding their assessments by about 12%, and General Residential parcels had prices exceeding their assessmens by about 18%.

Let's revisit the claims from above:

> Parcels with the Medium-Intensity Residential designation have been selling for over their official assessments

This is consistent with the data! Medium-Intensity Residential parcels sold for about 5% more than assessments overall for 2022 and 12% in March alone. What about the next claim?

> The difference between sale prices and assessments is greater for Medium-Intensity Residential than for other designations

This claim isn't supported here. While Medium-Intensity Residential parcels sold for over their assessment prices, this was also true for General Residential parcels, and it turns out that sale prices exceeeded assessments by an even higher proportion in General Residential. Sean wrote that

> This month's key phrase is "Medium Intensity Residential".

But maybe the unusual thing about Medium-Intensity Residential is that the recent sale prices of these parcels have been lower, relative to assessments, than the General Residential category that makes up the bulk of the city's residential land.

I've tried to evaluate these claims in a bit more detail, but there are many limitations to this quick analysis. Maybe the Medium-Intensity Residential properties that have been sold this year are in different neighborhoods than the General Residential properties; maybe the city assesses properties differently by land use designation in a biased way. Maybe there was a pre-existing difference in land or property values across designations. Maybe correcting for these potential differences, or others that I haven't thought of, would support the claim that Medium-Intensity Residential properties are suddenly uniquely expensive. But at least so far, it appears that sale prices are higher than assessments citywide, and if anything this is more true of General Residential than Medium-Intensity Residential in recent months.

It's also worth pointing out that we don't have much data yet. The Comprehensive Plan was passed in December 2021, and only 23 parcels zoned Medium-Intensity Residential have sold in 2022 (not including multi-parcel transactions or homes built after 2020). Meanwhile, the zoning rewrite is just starting: we haven't actually changed the rules yet. It's interesting to look at the early data we have, but realistically, it's going to take years to draw conclusions with any confidence.

Anyway, thanks to Sean for raising some interesting questions. If you're still reading, subscribe to his [substack](https://communityengagement.substack.com/) if you haven't already.

One last point: maybe this analysis has been skewed by a small number of unusual sales. Using medians rather than means should reduce the influence of outliers. And I've excluded sales that I don't think we should consider: specifically, properties that sold for $0 and properties that were sold as a package along with other properties. But maybe I missed something! So for completeness, here are the individual parcels that have sold in 2022, first Medium-Intensity Residential, then General Residential:

In [4]:
#collapse-hide
bq.query(
    """
    select
      sales.parcelnumb as parcelnumber,
      sales.saleamount,
      sales.saledate,
      assessments.totalvalue,
      sales.saleamount / assessments.totalvalue as saleassessmentratio,
      sales.streetnumb,
      sales.streetname,
      sales.unit,
    from `cvilledata.cville_open_data.real_estate_sales` sales
    join `cvilledata.cville_open_data.real_estate_all_assessments` assessments on sales.parcelnumb = assessments.parcelnumb and assessments.taxyear = 2022
    join `cvilledata.cville_plans_together.flum_202110` flum on sales.parcelnumb = flum.parcelnumb
    where sales.saledate >= '2022-01-01'
      and sales.saleamount > 0
      and sales.bookpage not in (
        select
          bookpage
        from `cvilledata.cville_open_data.real_estate_sales`
        where saledate >= '2022-01-01'
          and saleamount > 0
        group by bookpage
        having count(*) > 1
      )
      and flum.desig_oct = 'Medium-Intensity Residential'
    order by sales.saledate desc
    """
).result().to_dataframe()

,parcelnumber,saleamount,saledate,totalvalue,saleassessmentratio,streetnumb,streetname,unit
0,470041000,525000,2022-04-04,386300,1.359047,607,WATSON AVE,None
1,050013000,962500,2022-03-31,706300,1.362735,807,RUGBY RD,None
2,070003180,835000,2022-03-31,681700,1.224879,111,ALDERMAN RD,None
3,550049000,385000,2022-03-31,400900,0.960339,1338,CHESAPEAKE ST,None
4,43C041000,312500,2022-03-29,325200,0.960947,2239,BANBURY ST,None
5,21A131200,360700,2022-03-28,312500,1.154240,103,LONGWOOD DR,E
6,440028230,511000,2022-03-24,328400,1.556029,1435,WESTWOOD RD,None
7,41A101100,900000,2022-03-24,698700,1.288106,1616,MEADOWBROOK HEIGHTS RD,None
8,25A175000,365000,2022-03-21,338700,1.077650,159,BROOKWOOD DR,None
9,110076000,550000,2022-03-16,425900,1.291383,516,VALLEY RD,None


In [5]:
#collapse-hide
bq.query(
    """
    select
      sales.parcelnumb as parcelnumber,
      sales.saleamount,
      sales.saledate,
      assessments.totalvalue,
      sales.saleamount / assessments.totalvalue as saleassessmentratio,
      sales.streetnumb,
      sales.streetname,
      sales.unit,
    from `cvilledata.cville_open_data.real_estate_sales` sales
    join `cvilledata.cville_open_data.real_estate_all_assessments` assessments on sales.parcelnumb = assessments.parcelnumb and assessments.taxyear = 2022
    join `cvilledata.cville_plans_together.flum_202110` flum on sales.parcelnumb = flum.parcelnumb
    where sales.saledate >= '2022-01-01'
      and sales.saleamount > 0
      and sales.bookpage not in (
        select
          bookpage
        from `cvilledata.cville_open_data.real_estate_sales`
        where saledate >= '2022-01-01'
          and saleamount > 0
        group by bookpage
        having count(*) > 1
      )
      and flum.desig_oct = 'General Residential'
    order by sales.saledate desc
    """
).result().to_dataframe()

,parcelnumber,saleamount,saledate,totalvalue,saleassessmentratio,streetnumb,streetname,unit
0,230094200,360000,2022-04-07,372100,0.967482,512,PATON ST,None
1,41A006000,750000,2022-04-07,553800,1.354280,1631,BRANDYWINE DR,None
2,230090000,60000,2022-04-01,218200,0.274977,1306,KING ST,None
3,380045000,625000,2022-04-01,504100,1.239833,1610,BRUCE AVE,None
4,25A116101,550000,2022-04-01,471000,1.167728,948,RAYMOND RD,None
5,560114200,622431,2022-04-01,104500,5.956278,120,FRANKLIN ST,None
6,290164000,751100,2022-03-31,495300,1.516455,412,5TH ST SW,None
7,410043000,592000,2022-03-31,462400,1.280277,1427,GENTRY LN,None
8,050006000,1300000,2022-03-31,1154100,1.126419,1816,WINSTON RD,None
9,18A077B00,525000,2022-03-31,414600,1.266281,257,HUNTLEY AVE,None
